In [1]:
from __future__ import division, absolute_import

import sys
import random
import pickle

from sklearn import metrics
import pandas as pd
import numpy as np
import h5py
from plotnine import *

#root
absPath = '/home/angela3/imbalance_pcm_benchmark/'
sys.path.insert(0, absPath)

from src.Target import Target
from src.imbalance_functions import *

np.random.seed(8)
random.seed(8)

Using TensorFlow backend.


In [2]:
protein_type = "kinases"
nfolds = 10

group = '/activity'
table = "prot_comp"

#Loading maximum lengths of proteins and compounds
with open("".join((absPath, 'data/prot_max_len.pickle')), "rb") as input_file:
    max_len_prot = pickle.load(input_file)
#Defining protein dictionary    
instarget = Target("AAA")
prot_dict = instarget.predefining_dict()

with open("".join((absPath, "data/", protein_type, "/", protein_type, "_prots.pickle")), 'rb') as handle:
    unique_prots = pickle.load(handle)

In [3]:
protein_info = "".join((absPath, "data/", protein_type, "/prots_df.csv"))
prot_df = pd.read_csv(protein_info)
prot_df.drop(["Unnamed: 0"],axis=1, inplace=True)
prot_df.head()
prot_df["len_seq"] = prot_df.apply(len_seq, axis=1)

In [4]:
ratios_per_fold = []
for fold in range(nfolds):
    f_train = h5py.File("".join((absPath, "data/", protein_type, "/semi_resampling/",
                                 str(fold), "/compounds_activity.h5")), "r")
    f_test = h5py.File("".join((absPath, "data/", protein_type, "/semi_resampling/",
                                 str(fold), "/compounds_activity_test.h5")), "r")
    
    file_list = "".join((absPath, "data/", protein_type, "/semi_resampling/", str(fold), 
                         "/splitting_lists/splitting_", str(fold),"_list.pickle"))
    with open(file_list, "rb") as input_file:
        splitting_list = pickle.load(input_file)
    
    splitting_list[0].sort()
    splitting_list[1].sort()
    test_indices = range(len(f_test[group][table]))
    
    training_ratios = creating_ratios_list("training", fold, f_train, group, table, splitting_list[0],
                                           unique_prots)
    test_ratios = creating_ratios_list("test", fold, f_test, group, table, test_indices,
                                       unique_prots)
    
    pred_test_path = "".join((absPath, "data/", protein_type, "/semi_resampling/predictions/", 
                              str(fold), "/test.csv"))
    pred_test = pd.read_csv(pred_test_path)
    pred_test_ratios, pred_test_ = predictions_ratios_list(fold, pred_test, unique_prots)
    
    list_metrics = []
    for prot in unique_prots:
        dict_prot = computing_metrics_per_prot(prot, pred_test_)
        list_metrics.append(dict_prot)
        
    filt_list_metrics = []
    for val in list_metrics:
        if val != None:
            filt_list_metrics.append(val)
        
    ratios_df = converting_ratios_to_df(training_ratios, test_ratios, pred_test_ratios, filt_list_metrics,
                                        "semi_resampling", prot_df)
    ratios_df["fold"] = str(fold)
    ratios_per_fold.append(ratios_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156119 entries, 0 to 156118
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   DeepAffinity Protein ID  156119 non-null  object
 1   label                    156119 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13900 entries, 0 to 13899
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   DeepAffinity Protein ID  13900 non-null  object
 1   label                    13900 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 217.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13900 entries, 0 to 13899
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label                  

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/p

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/py

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


...

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.


/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/

Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
Only one class present in y_true.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  314 non-null    object 
 1   ratio_training           297 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------            

/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/angela3/venv/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` param

In [5]:
ratios_complete_df = pd.concat(ratios_per_fold)
ratios_seq = pd.merge(ratios_complete_df, prot_df, "left", on="DeepAffinity Protein ID")
print(ratios_seq.info())
print(ratios_seq.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2983 entries, 0 to 2982
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   DeepAffinity Protein ID  2983 non-null   object 
 1   ratio_training           2793 non-null   float64
 2   ratio_test               2983 non-null   float64
 3   ratio_test_predicted     2983 non-null   float64
 4   acc                      2983 non-null   float64
 5   auroc                    2246 non-null   float64
 6   f1                       2983 non-null   float64
 7   balanced_acc             2983 non-null   float64
 8   mcc                      2983 non-null   float64
 9   strategy                 2983 non-null   object 
 10  fold                     2983 non-null   object 
 11  Sequence                 2983 non-null   object 
 12  family                   2983 non-null   object 
 13  Uniprot ID               2983 non-null   object 
 14  n_interactions          

In [6]:
ratios_seq.to_csv("".join((absPath, "data/", protein_type, "/semi_resampling/results/ratios_df.csv")))